<a href="https://colab.research.google.com/github/oxygen0605/ImageClassification/blob/master/ResNet_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colaboratory環境の初期設定

## Google Driveにマウントしてマシンスペックを出力

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

!nvidia-smi > '/content/drive/My Drive/Colab Notebooks/Logs/machine_spec.txt'
!cat /proc/driver/nvidia/gpus/0000:00:04.0/information >> '/content/drive/My Drive/Colab Notebooks/Logs/machine_spec.txt'
!cat /etc/issue >> '/content/drive/My Drive/Colab Notebooks/Logs/machine_spec.txt'
!cat /proc/cpuinfo >> '/content/drive/My Drive/Colab Notebooks/Logs/machine_spec.txt'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!ls -al

total 20
drwxr-xr-x 1 root root 4096 Aug 25 15:07 .
drwxr-xr-x 1 root root 4096 Aug 25 14:55 ..
drwxr-xr-x 1 root root 4096 Aug 22 16:14 .config
drwx------ 3 root root 4096 Aug 25 15:07 drive
drwxr-xr-x 1 root root 4096 Aug 22 16:14 sample_data


# Deep CNN (CIFAR-10)

## モデルの生成

In [3]:
from keras.layers import Dense, Dropout, Activation, Flatten, Input, add
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras import Model

# this is a part of ResNet and WideResNet.
def rescell(data, filters, kernel_size, option=False):
    
    strides=(1,1)
    if option:
        strides=(2,2)
    
    x=Conv2D(filters=filters,
			 kernel_size=kernel_size,
			 strides=strides,
			 padding="same",
			 kernel_initializer='he_normal')(data)
    x=BatchNormalization()(x)
    x=Activation('relu')(x)

    # shortcut 
    data=Conv2D(filters=int(x.shape[3]), 
				kernel_size=(1,1), 
				strides=strides, 
				padding="same",
				kernel_initializer='he_normal')(data)

    x=Conv2D(filters=filters,
			 kernel_size=kernel_size,
			 strides=(1,1),
			 padding="same",
			 kernel_initializer='he_normal')(x)
    x=BatchNormalization()(x)

    # connnection
    x=add([x,data])

    x=Activation('relu')(x)

	
    return x

def ResNet34(input_shape, num_classes):
	input=Input(shape=input_shape)
  
	x=Conv2D(32,(7,7), padding="same", activation="relu",kernel_initializer='he_normal')(input)
	x=MaxPooling2D(pool_size=(2,2))(x)

	x=rescell(x,64,(3,3))
	x=rescell(x,64,(3,3))
	x=rescell(x,64,(3,3))

	x=rescell(x,128,(3,3),True)

	x=rescell(x,128,(3,3))
	x=rescell(x,128,(3,3))
	x=rescell(x,128,(3,3))

	x=rescell(x,256,(3,3),True)

	x=rescell(x,256,(3,3))
	x=rescell(x,256,(3,3))
	x=rescell(x,256,(3,3))
	x=rescell(x,256,(3,3))
	x=rescell(x,256,(3,3))

	x=rescell(x,512,(3,3),True)

	x=rescell(x,512,(3,3))
	x=rescell(x,512,(3,3))

	x=AveragePooling2D(pool_size=(int(x.shape[1]),int(x.shape[2])),strides=(2,2))(x)

	x=Flatten()(x)
	x=Dense(units=num_classes,kernel_initializer="he_normal",activation="softmax")(x)
	model=Model(inputs=input,outputs=[x])
  
	return model

Using TensorFlow backend.


## CIFAR10 データセットの用意

In [0]:
import keras
from keras.datasets import cifar10

class CIFAR10Dataset():
	def __init__(self):
		self.image_shape = (32, 32, 3)
		self.num_classes = 10
		
	def preprocess(self, data, label_data=False):
		if label_data:
			# conver class number to one-hot vector
			data = keras.utils.to_categorical(data, self.num_classes)
		
		else:
			data = data.astype("float32")
			data /= 255 #convert the value to 0 ~ 1 scale
			shape = (data.shape[0],) + self.image_shape
			data = data.reshape(shape)
			
		return data
	
	def get_batch(self):
		# x: data, y: lebel
		(x_train, y_train), (x_test, y_test) = cifar10.load_data()
		
		x_train, x_test = [self.preprocess(d) for d in [x_train, x_test]]
		y_train, y_test = [self.preprocess(d, label_data=True) for d in
					 [y_train, y_test]]
		
		return x_train, y_train, x_test, y_test

## TensorBoard用のログファイル生成関数

In [0]:
from __future__ import absolute_import
from __future__ import unicode_literals
from time import gmtime, strftime
from keras.callbacks import TensorBoard
import os


def make_tensorboard(set_dir_name=''):
    tictoc = strftime("%a_%d_%b_%Y_%H_%M_%S", gmtime())
    directory_name = tictoc
    log_dir = set_dir_name + '_' + directory_name
    os.mkdir(log_dir)
    tensorboard = TensorBoard(log_dir=log_dir, write_graph=True, )
    return tensorboard

## ImageDataGeneratorクラスの拡張
random crop
mix up
cutout
を実装する。

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

class ImageDataGeneratorEX(ImageDataGenerator):
	def __init__(self,
               featurewise_center=False,
               samplewise_center=False, 
               featurewise_std_normalization=False,
               samplewise_std_normalization=False,
               zca_whitening=False,
               zca_epsilon=1e-06,
               rotation_range=0.0,
               width_shift_range=0.0,
               height_shift_range=0.0,
               brightness_range=None,
               shear_range=0.0,
               zoom_range=0.0, 
               channel_shift_range=0.0,
               fill_mode='nearest',
               cval=0.0,
               horizontal_flip=False, 
               vertical_flip=False,
               rescale=None,
               preprocessing_function=None,
               data_format=None,
               validation_split=0.0, 
               random_crop=None,    # a new parameter for random crop
               mix_up_alpha=0.0,    # a new parameter for mix up
               cutout_mask_size=0   # a new parameter for cutout
              ):
    
		# 親クラスのコンストラクタ
		super().__init__(featurewise_center, samplewise_center, featurewise_std_normalization, samplewise_std_normalization, zca_whitening, zca_epsilon, rotation_range, width_shift_range, height_shift_range, brightness_range, shear_range, zoom_range, channel_shift_range, fill_mode, cval, horizontal_flip, vertical_flip, rescale, preprocessing_function, data_format, validation_split)
		# 拡張処理のパラメーター
		 # Mix-up
		assert mix_up_alpha >= 0.0
		self.mix_up_alpha = mix_up_alpha
		# Random Crop
		assert random_crop == None or len(random_crop) == 2
		self.random_crop_size = random_crop
		self.cutout_mask_size = cutout_mask_size
    
	# ランダムクロップ
    # 参考 https://jkjung-avt.github.io/keras-image-cropping/
	def random_crop(self, original_img):
        # Note: image_data_format is 'channel_last'
		assert original_img.shape[2] == 3
		if original_img.shape[0] < self.random_crop_size[0] or original_img.shape[1] < self.random_crop_size[1]:
			raise ValueError(f"Invalid random_crop_size : original = {original_img.shape}, crop_size = {self.random_crop_size}")
		height, width = original_img.shape[0], original_img.shape[1]
		dy, dx = self.random_crop_size
		x = np.random.randint(0, width - dx + 1)
		y = np.random.randint(0, height - dy + 1)
		return original_img[y:(y+dy), x:(x+dx), :]

    # Mix-up
    # 参考 https://qiita.com/yu4u/items/70aa007346ec73b7ff05
	def mix_up(self, X1, y1, X2, y2):
		assert X1.shape[0] == y1.shape[0] == X2.shape[0] == y2.shape[0]
		batch_size = X1.shape[0]
		l = np.random.beta(self.mix_up_alpha, self.mix_up_alpha, batch_size)
		X_l = l.reshape(batch_size, 1, 1, 1)
		y_l = l.reshape(batch_size, 1)
		X = X1 * X_l + X2 * (1-X_l)
		y = y1 * y_l + y2 * (1-y_l)
		return X, y
    
	def cutout(self, x, y):
		return np.array(list(map(self._cutout, x))), y

	def _cutout(self, image_origin):
		# 最後に使うfill()は元の画像を書き換えるので、コピーしておく
		img = np.copy(image_origin)
		mask_value = img.mean()
		# 乱数固定(flowでseed固定したら必要ないかも)

		h, w, _ = img.shape
		# マスクをかける場所のtop, leftをランダムに決める
		# はみ出すことを許すので、0以上ではなく負の値もとる(最大mask_size // 2はみ出す)
		top = np.random.randint(0 - self.cutout_mask_size // 2, h - self.cutout_mask_size)
		left = np.random.randint(0 - self.cutout_mask_size // 2, w - self.cutout_mask_size)
		bottom = top + self.cutout_mask_size
		right = left + self.cutout_mask_size

		# はみ出した場合の処理
		if top < 0:
			top = 0
		if left < 0:
			left = 0

		# マスク部分の画素値を平均値で埋める
		img[top:bottom, left:right, :].fill(mask_value)
		return img


	def flow(self, 
			 x, y=None, 
			 batch_size=32, 
			 shuffle=True,
			 sample_weight=None,
			 seed=None, 
			 save_to_dir=None, 
			 save_prefix='', 
			 save_format='png', 
			 subset=None
		):
		
		batches = super().flow(x, y, batch_size, shuffle, sample_weight, seed, save_to_dir, save_prefix, save_format, subset)
		# 拡張処理
		while True:
			batch_x, batch_y = next(batches)
			
			# mix up
			if self.mix_up_alpha > 0:
				while True:
					batch_x_2, batch_y_2 = next(batches)
					m1, m2 = batch_x.shape[0], batch_x_2.shape[0]
					
					if m1 < m2:
						batch_x_2 = batch_x_2[:m1]
						batch_y_2 = batch_y_2[:m1]
						break
					elif m1 == m2:
						break
				batch_x, batch_y = self.mix_up(batch_x, batch_y, batch_x_2, batch_y_2)
			
			# Random crop
			if self.random_crop_size is not None:
				x = np.zeros((batch_x.shape[0], self.random_crop_size[0], self.random_crop_size[1], 3))
				for i in range(batch_x.shape[0]):
					x[i] = self.random_crop(batch_x[i])
				batch_x = x
			
			if self.cutout_mask_size > 0:
				batch_x, batch_y = self.cutout(batch_x, batch_y)
			
			
			yield (batch_x, batch_y)

## Training, Evaluation用クラスの定義

In [0]:
import os
from keras.models import load_model
from keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler
from keras.optimizers import RMSprop, Adam, Nadam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import numpy as np

from __future__ import absolute_import
from __future__ import unicode_literals
from time import gmtime, strftime
import os


def make_tensorboard(set_dir_name=''):
    tictoc = strftime("%a_%d_%b_%Y_%H_%M_%S", gmtime())
    directory_name = tictoc
    log_dir = set_dir_name + '_' + directory_name
    os.mkdir(log_dir)
    tensorboard = TensorBoard(log_dir=log_dir, write_graph=True, )
    return tensorboard

class Trainer():
	
	def __init__(self, model, loss, optimizer, logdir = './'):
		self._target = model
		self._target.compile(
				loss=loss, optimizer=optimizer, metrics=["accuracy"]
				)
		self.verbose = 1 # visualize progress bar: 0(OFF), 1(On), 2(On:each data) 
		#self.log_dir = os.path.join(os.path.dirname(__file__), logdir)
		self.log_dir = os.path.join(logdir)
		self.model_file_name = "model_file.hdf5"
	
	def train_for_tuning_test_data(self, 
            x_train, y_train, x_test, y_test, batch_size, epochs, lr_scheduler):
		datagen = ImageDataGeneratorEX(
			      featurewise_center=False,            # set input mean to 0 over the dataset
            samplewise_center=False,             # set each sample mean to 0
            featurewise_std_normalization=False, # divide inputs by std
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,                 # apply ZCA whitening
            rotation_range=20,                   # randomly rotate images in the range (0~180)
            width_shift_range=0.2,               # randomly shift images horizontally
            height_shift_range=0.2,              # randomly shift images vertically
            zoom_range = 0.2,
            channel_shift_range = 0.2,
            horizontal_flip=True,                # randomly flip images
            vertical_flip=False,                 # randomly flip images
            random_crop=None,
			      mix_up_alpha=0.0,
			      cutout_mask_size=0
		)
        
    # training (validation dataはデータ拡張はしない)
		model_path = os.path.join(self.log_dir, self.model_file_name)
		self._target.fit_generator(
            generator        = datagen.flow(x_train,y_train, batch_size,seed=0),
            steps_per_epoch  = x_train.shape[0] // batch_size,
            epochs           = epochs,
            validation_data  = ImageDataGenerator().flow(x_test,y_test, batch_size),
			      validation_steps = x_test.shape[0] // batch_size,
            callbacks=[
                LearningRateScheduler(lr_scheduler),
                make_tensorboard(set_dir_name=self.log_dir),
                ModelCheckpoint(model_path, save_best_only=True,monitor='val_acc',mode='max')
            ],
            verbose = self.verbose,
            use_multiprocessing=True,
            #workers = 4
        )
		

class Evaluator():
    
    def __init__(self, result_file_path="./prediction_result.csv"):
        self.result_file_path="./prediction_result.csv"
        
    def simple_evaluate(self, model, x_test, label):
        print("start evaluation...")
        score = model.evaluate(x_test, y_test, verbose=1)
        print("Test loss:", score[0])
        print("Test accuracy:", score[1])
        return score
    
    def tta_evaluate(self, model, x_test, label, batch_size = 2500, tta_epochs = 2):
        print("batch size (TTA): "+str(batch_size))
        print("epochs (TTA): "+str(tta_epochs))
        tta = TTA()
        tta_pred = tta.predict(model, x_test, batch_size, epochs = tta_epochs)
        print("Test accuracy(TTA): ",end = "")
        print( accuracy_score( np.argmax(tta_pred,axis = 1) , np.argmax(label,axis = 1)))
        return tta_pred

## 学習率減衰

In [0]:
def learning_rate_schedule_for_Adam(epoch):
	lr = 0.001
	if(epoch >= 200): lr = 0.0002 
	if(epoch >= 275): lr = 0.00002
	return lr

def learning_rate_schedule_for_SGD(epoch):
	lr = 0.1
	if(epoch >= 100): lr = 0.01 
	if(epoch >= 200): lr = 0.001
	return lr

## 実行

In [0]:
from keras.optimizers import Adam, SGD
from keras.models import load_model


# create dataset
dataset = CIFAR10Dataset()
x_train, y_train, x_test, y_test = dataset.get_batch()


save_dir='/content/drive/My Drive/Colab Notebooks/Logs/ResNet_1/'

# create model
model = ResNet34(dataset.image_shape, dataset.num_classes)

#model = load_model(save_dir+'model_file.hdf5')
model.summary()

# train the model
#trainer = Trainer(model, loss="categorical_crossentropy", optimizer=SGD(lr=0.1, momentum=0.9, decay=0.0005, nesterov=True), logdir=save_dir)
trainer = Trainer(model, loss="categorical_crossentropy", optimizer=Adam(), logdir=save_dir)
trainer.train_for_tuning_test_data(
            x_train, y_train, x_test, y_test, batch_size=128, epochs=300, 
            lr_scheduler=learning_rate_schedule_for_Adam)


# bestなモデルをロードする
model = load_model(save_dir+trainer.model_file_name)
#model = load_model(save_dir+"cnn_best_acc_model.hdf5")

# show result
evaluator = Evaluator()
score = evaluator.simple_evaluate(model, x_test, y_test)

170500096/170498071 [==============================] - 6s 0us/step


W0825 15:07:55.736677 139793094752128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



##  Test Time Augmentation（TTA）を用いた推論 

In [0]:
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

class TTA:
    
    #test_time_augmentation
    #batch_sizeは，test_sizeの約数!!!
    def predict(self, model, x_test, batch_size ,epochs = 10):
        
        # Augmentation用generatorによるデータセットの作成
        data_flow = self.generator(x_test, batch_size)
        
        test_size = x_test.shape[0]
        pred = np.zeros(shape = (test_size,10), dtype = float)
        
        step_per_epoch = test_size //batch_size
        for epoch in range(epochs):
            print( 'epoch: ' + str(epoch+1)+'/'+str(epochs))
            for step in range(step_per_epoch):
                #print( 'step: ' + str(step+1)+'/'+str(step_per_epoch))
                sta = batch_size * step
                end = sta + batch_size
                tmp_x = data_flow.__next__()
                pred[sta:end] += model.predict(tmp_x)        
        return pred / epochs
    
    
    def generator(self, x_test,batch_size):
        return ImageDataGeneratorEX(
                    rotation_range = 10,
                    horizontal_flip = True,
                    height_shift_range = 0.1,
                    width_shift_range = 0.1,
                    zoom_range = 0.1,
                    channel_shift_range = 0.1,
            			  #random_crop=None,
			              #mix_up_alpha=0.2,
			              #cutout_mask_size=16
                ).flow(x_test,batch_size = batch_size,shuffle = False, seed=756) #756 9447

      
# show result
evaluator = Evaluator()
score = evaluator.tta_evaluate(model, x_test, y_test, batch_size = 500, tta_epochs = 50)